In [1]:
import copy
import itertools as its
import math
import os
import pathlib
import re
import sys
from typing import Dict, List, Optional, Tuple, Union
from collections import Counter, defaultdict, deque

import networkx as nx
import numpy as np
import pandas as pd
from IPython.display import clear_output
from matplotlib import pyplot as plt

from aoc import sim_new as sim, testing, util

twopi = 2 * math.pi

%matplotlib inline

INPUT_PATH = pathlib.Path('..') / 'input' / 'dec20.txt'

In [2]:
WALL = '#'
EMPTY = '.'

OUTER = 0
INNER = 1

START_NAME = 'AA'
END_NAME = 'ZZ'

def read_maze(text: str) -> Dict[Tuple[int, int], str]:
    maze = {}
    maze_as_list = [[c for c in line.rstrip()] for line in text.rstrip().split('\n')]
    for y, line in enumerate(maze_as_list):
        for x, c in enumerate(line):
            pos = (x, y)
            if pos in maze:
                continue
            if c in [WALL, EMPTY]:
                maze[pos] = c
            elif c == ' ':
                continue
            else:
                # We're some sort of label
                up = (x, y - 1)
                down = (x, y + 1)
                left = (x - 1, y)
                right = (x + 1, y)
                if up in maze and maze[up] not in [WALL, EMPTY]:
                    cup = maze[up]
                    del maze[up]
                    # If there's something present to the down, place at pos else at up
                    place = (x, y - 2)
                    try:
                        if maze_as_list[down[1]][down[0]] == EMPTY:
                            place = down
                    except: pass
                    maze[place] = cup + c
                elif left in maze and maze[left] not in [WALL, EMPTY]:
                    cup = maze[left]
                    del maze[left]
                    place = (x - 2, y)
                    try:
                        if maze_as_list[right[1]][right[0]] == EMPTY:
                            place = (x + 1, y)
                    except: pass
                    maze[place] = cup + c
                else:
                    # Just leave here for future read
                    maze[pos] = c
    return maze

In [3]:
def door_orientations(maze):
    maze = {key: val for key, val in maze.items() if val != WALL}
    door_coords = [key for key, val in maze.items() if val != EMPTY]
    left_x = min(key[0] for key in door_coords)
    right_x = max(key[0] for key in door_coords)
    top_y = min(key[1] for key in door_coords)
    bottom_y = max(key[1] for key in door_coords)

    return {
        key: OUTER if key[0] in [left_x, right_x] or key[1] in [top_y, bottom_y] else INNER
        for key in door_coords
    }

In [4]:
def maze_to_graph(maze, depth: int = 0):
    g = nx.Graph()
    
    maze = {key: val for key, val in maze.items() if val != WALL}
    door_to_coords = defaultdict(list)
    for key, val in maze.items():
        if val != EMPTY:
            door_to_coords[val].append(key)

    start_pos = door_to_coords[START_NAME][0]
    del door_to_coords[START_NAME]
    end_pos = door_to_coords[END_NAME][0]
    del door_to_coords[END_NAME]
    
    # Just test our orientation function
    orientations = door_orientations(maze)
    for door, keys in door_to_coords.items():
        assert sum(orientations[key] for key in keys) == 1
    
    # Make sure we have two of each door
    assert all(len(val) == 2 for val in door_to_coords.values())

    # Add each level, the basic connections
    for d in range(depth + 1):
        for key, val in maze.items():
            for to_pos in util.four_ways(*key):
                if to_pos in maze:
                    g.add_node((key[0], key[1], d))
                    g.add_node((to_pos[0], to_pos[1], d))
                    g.add_edge((key[0], key[1], d), (to_pos[0], to_pos[1], d))

    # Add the doors.
    
    # If depth == 0, just connect up everything so everything always finishes
    if not depth:
        for door, coords in door_to_coords.items():
            left, right = coords
            g.add_edge((left[0], left[1], depth), (right[0], right[1], depth))

    # For each other depth, connect inner doors to outer doors one level deeper
    for d in range(depth):
        for door, coords in door_to_coords.items():
            left, right = coords
            # Set orientation correctly; right = OUTER, left = INNER
            if orientations[left] != INNER:
                left, right = right, left
            # The INNER door connects to the OUTER door one level down
            g.add_edge((left[0], left[1], d), (right[0], right[1], d + 1))

    return g, (start_pos[0], start_pos[1], 0), (end_pos[0], end_pos[1], 0)

In [5]:
maze = read_maze(INPUT_PATH.read_text())
graph, start_pos, end_pos = maze_to_graph(maze)

In [6]:
print(f'The answer to part 1 is {nx.shortest_path_length(graph, start_pos, end_pos)}')

The answer to part 1 is 686


In [7]:
test_maze = """
             Z L X W       C                 
             Z P Q B       K                 
  ###########.#.#.#.#######.###############  
  #...#.......#.#.......#.#.......#.#.#...#  
  ###.#.#.#.#.#.#.#.###.#.#.#######.#.#.###  
  #.#...#.#.#...#.#.#...#...#...#.#.......#  
  #.###.#######.###.###.#.###.###.#.#######  
  #...#.......#.#...#...#.............#...#  
  #.#########.#######.#.#######.#######.###  
  #...#.#    F       R I       Z    #.#.#.#  
  #.###.#    D       E C       H    #.#.#.#  
  #.#...#                           #...#.#  
  #.###.#                           #.###.#  
  #.#....OA                       WB..#.#..ZH
  #.###.#                           #.#.#.#  
CJ......#                           #.....#  
  #######                           #######  
  #.#....CK                         #......IC
  #.###.#                           #.###.#  
  #.....#                           #...#.#  
  ###.###                           #.#.#.#  
XF....#.#                         RF..#.#.#  
  #####.#                           #######  
  #......CJ                       NM..#...#  
  ###.#.#                           #.###.#  
RE....#.#                           #......RF
  ###.###        X   X       L      #.#.#.#  
  #.....#        F   Q       P      #.#.#.#  
  ###.###########.###.#######.#########.###  
  #.....#...#.....#.......#...#.....#.#...#  
  #####.#.###.#######.#######.###.###.#.#.#  
  #.......#.......#.#.#.#.#...#...#...#.#.#  
  #####.###.#####.#.#.#.#.###.###.#.###.###  
  #.......#.....#.#...#...............#...#  
  #############.#.#.###.###################  
               A O F   N                     
               A A D   M                     
"""

In [8]:
def recursive_find_path(maze):
    depth = 1
    while True:
        graph, start_pos, end_pos = maze_to_graph(maze, depth=depth)
        try:
            path_length = nx.shortest_path_length(graph, start_pos, end_pos)
            return path_length
        except:
            depth += 10


In [9]:
maze = read_maze(test_maze)
assert recursive_find_path(maze) == 396

In [10]:
maze = read_maze(INPUT_PATH.read_text())
print(f'The answer to part 2 is {recursive_find_path(maze)}')

The answer to part 2 is 8384
